In [1]:
import sys
!{sys.executable} -m pip install pandas

     |████████████████████████████████| 10.5MB 5.9MB/s eta 0:00:01    |████████▌                       | 2.8MB 5.9MB/s eta 0:00:02


In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sqlContext = SQLContext(sc)

df = sqlContext.read.load("gs://bi-bigdata-ws/sales_all.csv",
                         format="com.databricks.spark.csv",
                         header="true",
                         inferSchema="true")

# how many distinct products
df

DataFrame[date: timestamp, WeekdayNo: int, Weekday: string, HolidayFlag: boolean, SchoolHolidayFlag: boolean, Sales: double]

In [43]:
weekdays=df.select('Weekday').distinct().toPandas()

In [7]:
import pandas as pd

In [44]:
weekdays

,Weekday
0,Thur
1,Sun
2,Mon
3,Sat
4,Wed
5,Tue
6,Fri


In [39]:
df.groupBy('Weekday').sum('Sales').toPandas().head(7)

,Weekday,sum(Sales)
0,Thur,25563.0
1,Sun,29641.0
2,Mon,31278.0
3,Sat,18769.0
4,Wed,21459.0
5,Tue,23711.0
6,Fri,27087.0


In [58]:
df.registerTempTable('sales')
spark.sql("SELECT sum(Sales) from sales GROUP BY Weekday").toPandas()

,sum(Sales)
0,25563.0
1,29641.0
2,31278.0
3,18769.0
4,21459.0
5,23711.0
6,27087.0


In [45]:
from pyspark.sql import functions as F

,Summe,Mean
0,25563.0,92.956364
1,29641.0,107.785455
2,31278.0,113.738182
3,18769.0,68.500000
4,21459.0,78.032727
5,23711.0,86.221818
6,27087.0,98.857664


In [69]:
df.groupBy('Weekday').agg(F.sum('Sales'),F.mean('Sales')).toPandas()

,Weekday,sum(Sales),avg(Sales)
0,Thur,25563.0,92.956364
1,Sun,29641.0,107.785455
2,Mon,31278.0,113.738182
3,Sat,18769.0,68.500000
4,Wed,21459.0,78.032727
5,Tue,23711.0,86.221818
6,Fri,27087.0,98.857664


In [74]:
df.registerTempTable('sales')
spark.sql("SELECT Weekday, sum(Sales) as Summe ,mean(Sales) as Mean FROM sales GROUP BY Weekday").toPandas()

,Weekday,Summe,Mean
0,Thur,25563.0,92.956364
1,Sun,29641.0,107.785455
2,Mon,31278.0,113.738182
3,Sat,18769.0,68.500000
4,Wed,21459.0,78.032727
5,Tue,23711.0,86.221818
6,Fri,27087.0,98.857664


In [75]:
from pyspark.ml.feature import OneHotEncoder

In [79]:
encoder = OneHotEncoder(inputCol='WeekdayNo',outputCol='WeekdayNoEncoded')
encoded_df=encoder.transform(df).select('WeekdayNoEncoded','HolidayFlag','SchoolHolidayFlag','Sales')
encoded_df.toPandas()

,WeekdayNoEncoded,HolidayFlag,SchoolHolidayFlag,Sales
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",True,True,117.0
1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",False,True,106.0
2,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",False,True,82.0
3,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",False,True,70.0
4,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",False,True,84.0
...,...,...,...,...
1918,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",False,False,107.0
1919,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",False,False,113.0
1920,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",False,False,87.0
1921,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",False,False,69.0


Pyspark braucht genau einen Vector mit den feature Infos --> Daher müssen holidayflag usw auch in den vector

In [83]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols=['WeekdayNoEncoded','HolidayFlag','SchoolHolidayFlag'],outputCol='features')
encoded_df_prep=va.transform(encoded_df).select('features','Sales')
encoded_df_prep.toPandas().head()

,features,Sales
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0)",117.0
1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",106.0
2,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",82.0
3,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0)",70.0
4,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)",84.0


In [84]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features',labelCol='Sales')
model=lr.fit(encoded_df_prep)

In [85]:
modelsummary =model.summary

In [86]:
modelsummary.meanAbsoluteError

2.475414525923565

In [87]:
model.transform(encoded_df_prep).toPandas()

,features,Sales,prediction
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0)",117.0,115.226003
1,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",106.0,107.348874
2,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",82.0,80.377942
3,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0)",70.0,71.830944
4,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)",84.0,86.168674
...,...,...,...
1918,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",107.0,108.956033
1919,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",113.0,115.028418
1920,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",87.0,88.057486
1921,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",69.0,79.510488
